In [1]:
import pandas as pd
import json


filename = "place_data.json"

In [2]:
records = []


def try_parse_json(string):
    try:
        return json.loads(string), None
    except json.JSONDecodeError as e:
        return None, e


with open(filename, "r") as file:
    buffer = ""
    for line in file:
        buffer += line.strip()
        parsed_json, error = try_parse_json(buffer)
        if parsed_json:
            buffer = ""  # Clear buffer if successful parse
            if parsed_json["op"] == "mcm":
                pt = parsed_json["pt"]
                for mc_item in parsed_json.get("mc", []):
                    market_id = mc_item["id"]
                    for rc in mc_item.get("rc", []):
                        if "batb" in rc and len(rc["batb"]) > 1:
                            best_available_to_back = rc["batb"][1][1]
                        else:
                            best_available_to_back = None
                        ltp = rc.get("ltp")
                        tv = rc.get("tv")
                        horse_id = rc["id"]
                        records.append(
                            {
                                "timestamp_unix": pt,
                                "market_id": market_id,
                                "horse_id": horse_id,
                                "best_available_to_back": best_available_to_back,
                                "last_traded_price": ltp,
                                "traded_volume": tv,
                            }
                        )
        elif error and "Extra data" in str(
            error
        ):  # Possible end of one object and start of another
            first_part, _, buffer = buffer.partition("}{")
            # Assume the split is between separate JSON objects
            buffer = "{" + buffer
            first_part += "}"
            try_parse_json(first_part)  # Retry parsing the first part
        # Handle other types of JSONDecodeError if necessary

In [3]:
# Create a DataFrame from the records
df = pd.DataFrame(records)

In [4]:
df.tail(1000)

,timestamp_unix,market_id,horse_id,best_available_to_back,last_traded_price,traded_volume
22151,1580573797519,1.168096597,23405261,NaN,11.00,1007.59
22152,1580573797519,1.168096597,21848159,2.54,3.10,9996.87
22153,1580573797519,1.168096597,17596196,1.49,1.78,9070.90
22154,1580573798479,1.168096597,22023487,1.70,3.40,3270.98
22155,1580573798479,1.168096597,22046306,NaN,2.66,3217.77
...,...,...,...,...,...,...
23146,1580573984225,1.168096597,14765349,1.27,5.80,6824.50
23147,1580573984225,1.168096597,22972635,1.14,4.50,1890.92
23148,1580573984225,1.168096597,22101589,NaN,1.03,11795.21
23149,1580573984225,1.168096597,23405261,1.10,3.70,1039.41


In [5]:
# describe the dataframe

df.describe()

,timestamp_unix,horse_id,best_available_to_back,last_traded_price,traded_volume
count,2.315100e+04,2.315100e+04,9942.000000,23151.000000,23151.000000
mean,1.580558e+12,2.018752e+07,3.108045,3.394366,1236.235645
std,1.809309e+07,3.020924e+06,1.683398,2.005571,2129.182236
min,1.580495e+12,1.476535e+07,1.010000,0.000000,0.000000
25%,1.580550e+12,1.759620e+07,1.950000,2.120000,102.650000
50%,1.580565e+12,2.202349e+07,2.680000,2.840000,420.180000
75%,1.580573e+12,2.210159e+07,3.750000,4.100000,1186.690000
max,1.580574e+12,2.340526e+07,13.500000,16.000000,11795.210000


In [6]:
df.shape

(23151, 6)

In [8]:
# save the dataframe to a csv file

df.to_csv("csv_files/place_data.csv", index=False)